In [29]:
!pip install gradio

In [30]:
import yfinance as yf
import numpy as np
import pandas as pd
import gradio as gr
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [31]:
# 1. Fetch stock data
import yfinance as yf

def get_stock_data(ticker, period='5y'):
    data = yf.download(ticker, period=period)
    print(f"Top 5 rows of {ticker}:")
    print(data.head())
    print("\n" + "="*50 + "\n")
    return data[['Close']]

stock_options = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
stock_data = {}

for stock in stock_options:
    stock_data[stock] = get_stock_data(stock)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Top 5 rows of AAPL:
Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2020-04-15  69.025200  69.486289  68.103022  68.532562  131154400
2020-04-16  69.573647  69.940094  68.520421  69.741096  157125200
2020-04-17  68.629623  69.636747  67.188109  69.088290  215250000
2020-04-20  67.205101  68.357826  67.185690  67.452638  130015200
2020-04-21  65.127769  67.282760  64.414293  67.047361  180991600


Top 5 rows of MSFT:
Price            Close        High         Low        Open    Volume
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT
Date                                                                
2020-04-15  164.519882  166.137517  161.992931  163.868993  40940800
2020-04-16  169.458908  169.688636  165.496189  166.836248  50479600
2020-04-17  170.952133  172.292178  168.339025  171.813588  52765600
2020-04-20  167.

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Top 5 rows of GOOGL:
Price           Close       High        Low       Open    Volume
Ticker          GOOGL      GOOGL      GOOGL      GOOGL     GOOGL
Date                                                            
2020-04-15  62.565876  63.452138  61.406418  62.028942  42236000
2020-04-16  62.572342  63.365053  61.615419  63.055532  57896000
2020-04-17  63.645714  64.193096  62.998805  63.780070  51050000
2020-04-20  62.757462  63.537234  62.523082  63.192383  35292000
2020-04-21  60.319614  62.202610  59.963316  61.839846  49648000


Top 5 rows of AMZN:
Price            Close        High         Low        Open     Volume
Ticker            AMZN        AMZN        AMZN        AMZN       AMZN
Date                                                                 
2020-04-15  115.384003  116.668503  112.250000  112.884003  137332000
2020-04-16  120.409500  123.050003  116.750000  117.300003  240764000
2020-04-17  118.750000  120.000000  115.801003  118.616501  158600000
2020-04-20  119.6

In [32]:
# 2. Preprocess data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler

In [33]:
# 3. Create sequences
def create_sequences(data, time_steps=60):
    x, y = [], []
    for i in range(len(data) - time_steps - 1):
        x.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(x), np.array(y)

In [34]:
# 4. Build LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, input_shape=input_shape, return_sequences=True),
        LSTM(50, return_sequences=False),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [35]:
# 5. Train the model
def train_model(ticker):
    data = get_stock_data(ticker)
    data_scaled, scaler = preprocess_data(data)
    x, y = create_sequences(data_scaled, time_steps=60)
    x = x.reshape(x.shape[0], x.shape[1], 1)

    model = build_lstm_model((x.shape[1], 1))
    model.fit(x, y, epochs=10, batch_size=16, verbose=1)
    return model, scaler, data

In [36]:
# 6. Predict future stock prices
def predict_future(ticker, days=30):
    model, scaler, data = train_model(ticker)
    data_scaled, _ = preprocess_data(data)

    last_60_days = data_scaled[-60:]
    X_input = last_60_days.reshape(1, 60, 1)
    predictions = []

    for _ in range(days):
        prediction = model.predict(X_input, verbose=0)[0][0]
        predictions.append(prediction)
        X_input = np.append(X_input[:, 1:, :], [[[prediction]]], axis=1)

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    return predictions.flatten()

In [37]:
# 7. Plot predictions
def plot_predictions(predictions):
    plt.figure(figsize=(10, 4))
    plt.plot(predictions, label='Predicted Price')
    plt.xlabel('Days')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.tight_layout()
    plt.savefig('predictions.png')
    return 'predictions.png'

In [38]:
# 8. Gradio interface
stock_options = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']

def gradio_interface(ticker, days):
    predictions = predict_future(ticker, days)
    plot_path = plot_predictions(predictions)
    return plot_path

demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Dropdown(choices=stock_options, label='Select Stock Ticker'),
        gr.Slider(1, 30, step=1, label='Days to Predict')
    ],
    outputs=gr.Image(label='Predicted Stock Prices'),
    title="Stock Price Prediction",
    description="Enter a stock ticker and the number of days to predict."
)

In [39]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://76d734945b836b35cc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
